## Testing Search Problem Algorithm
In this notebook, we will test for different scenarios using the custom defined classes and solver algorithm in `search_problem.py`.
We will set up the search problem varying the fiollowing parameters:
* n_runway
* max_delay
* duration of disruption (by chaning the paramter `resume_hour` and/or `resume_min`)
* Number of flights to consider (by changing the df passed into the `search_pronble` class)

By default, the `resschedule_problem` class defines the problem with the following parameters:
* n_runway = 1
* disruption_dur = 60 (in minutes)
* duration of time slot = 5 (in minutes)
* Maximum delay permissible before a flight has to be diverted = 120


In [53]:
import pandas as pd 
import numpy as np
import itertools
# _________________________________
# search problem
class reschedule_problem:

    def __init__(self,df, n_runway = 1, disruption_dur = 60,\
                timeslot_dur = 5, max_delay = 120,divert_penalty = 1):
        # customized attributes of the problem
        self.df = df
        self.move = df['code']
        self.n_runway = n_runway
        self.resumetime = self.df.iloc[0,0] + pd.Timedelta(minutes = disruption_dur)
        self.hour = self.resumetime.hour
        self.min = self.resumetime.minute
        self.time_slot_duration = timeslot_dur
        self.max_delay = max_delay
        self.divert_penalty = divert_penalty # referenced in self.compute_util
        # parsing the attribute for more useful format
        self.date = self.df.loc[0,'time_sch'].day
        self.month = self.df.loc[0,'time_sch'].month
        self.year = self.df.loc[0,'time_sch'].year
        self.initial = pd.DataFrame(columns = ["time_new","util"], index= []) # to store the solution
        self.solution = None

    def actions(self, state:pd.DataFrame):
        """return a list of n_runway flight(s) that has/have is expected to land
        and is not assigned a time slot under the state; and one list of flight
        that has to be diverted"""
        # get the list of unassigned flight where the time_sch is (over)due under current time slot 
        assigned_flight = list([flight for flight in state.index])
        unassigned_flight = self.df.query("code not in @assigned_flight").copy()
        # subset the flights that are not in time yet / not diverted
        year, month, date, hour, min = self.parse_state_time(state)
        current_time = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
        min_time = pd.to_datetime(f"{year} {month} {date} {hour}:{min}") - pd.Timedelta(minutes = self.max_delay)
        # create boolean columns for subsetting diverted flights and those to schedule
        unassigned_flight['divert'] = unassigned_flight.apply(lambda flight: flight['time_sch'] < min_time,axis = 1)
        unassigned_flight['to_sch'] = unassigned_flight.apply(lambda flight: flight['time_sch'] <=current_time  ,axis = 1)

        # return the pd.Series of unassigned flight that has passed its schedule time
        flight_to_assign = unassigned_flight[unassigned_flight['to_sch'] & ~(unassigned_flight['divert'])]['code']
        # return the combination of flights (namely when n_runway > 1)
        flight_to_assign = [comb for n in range(1, self.n_runway+1) for comb in itertools.combinations(flight_to_assign, n)]
        # flight to divert
        flight_diverted = unassigned_flight[unassigned_flight['divert']]['code']

        return flight_to_assign, flight_diverted # pd.Series
        
    def parse_state_time(self,state:pd.DataFrame):
        """Use to get the value in the schedule and 
        parse the time of the next time slot"""
        if len(state) <=0: 
            year = self.year
            month = self.month
            date = self.date
            hour = self.hour
            min = self.min

        else:
            timestr = state['time_new'].iloc[-1] # get the time_sch
            year, month, date, time = timestr.split(" ")
            hour , min = time.split(":")
            # get the current time
            min = int(min)
            min += self.time_slot_duration
            if min // 60 <= 1:
                hour = int(hour) + min // 60
                min = min % 60
                if hour // 24 <= 1:
                    date = int(date) + hour // 24
                    hour = hour % 24
                
        return year, month, date, hour,min
    
    def result(self, state:pd.DataFrame, flight_diverted:list, flights:list):
        """return the state in a form of dictionary that is the result of a given move"""
        # parse the time data
        year, month, date, hour, min = self.parse_state_time(state)

        if len(flights) != 0:
            for fl in flights:
                time_sch = f"{year} {month} {date} {hour:02d}:{min:02d}"
                # get the utility
                util = self.compute_util(fl, year, month, date, hour, min)
                # print(f"Scheudling {flights} for {time_sch}")
                state.loc[fl] = [time_sch, util]
        elif not self.goal_test(state):
            # no flight can be assigned to the time slot
            time_sch = f"{year} {month} {date} {hour:02d}:{min:02d}"
            fl ='no flight to assign'
            state.loc[fl] = [time_sch, 0]
    
        # add the diverted flight into the state
        for fl in flight_diverted:
            util = self.compute_util(fl,1970, 0,0,0,0)
            state.loc[fl] = ["1970 01 01 00:00", util]
            # print(f"Diverting {flights}")

        # sort the state df by the new scheduled time
        state = state.sort_values("time_new")

        return state # pd.DataFrame
    
    def compute_util(self, flcode:str, year, month, date, hour, min):
        """Compute the utility of a given rescheduled flight. This is defined
        as the time difference between the original scheduled time and the 
        new scheduled time"""
        if flcode is None:
            return 0 
        elif year != 1970:
            time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            # compute the time delayed
            delay = time_sch_org - time_sch_new
            delay = delay.reset_index(drop = True)[0].total_seconds() / 60
            return delay
        elif year == 1970:
            # compute utility of diverted flight
            delay = - self.max_delay * self.divert_penalty
            return delay
    
    def utility(self, state:pd.DataFrame):
        """Compute aggregate utility of a given state"""
        agg_util = state['util'].sum()
        return agg_util
        
    def goal_test(self, state:pd.DataFrame):
        """return True if the state is terminal"""
        flight_assigned = [flight for flight in state.index] 
        if len(flight_assigned) == len(self.df):
        # !!! this is not a robust way since time slot can be none
            return True
        
    def solve(self, solver_algo):
        self.solution = solver_algo(self)   
        if self.solution is None:
            print("No solution is returned")
        
    def display(self):
        """ Display the self.solution that is of type pandas dataframe
        to compare the solution with the set up that we try to solve 
        """
        if self.solution is None:
            print("The problem has not been solved. Pass a solving algorithm to the .solve method")
            raise NotImplementedError
        elif type(self.solution) != pd.core.frame.DataFrame:
            print("""THe solution returned by the algorithm cannot be parsed because it is 
                  not of type pd.core.frame.DataFrame""")
        
        display_df = self.df.copy()
        display_df = pd.merge(self.df[['code','time_sch','pass_load']], self.solution, left_on= "code", right_index = True)
        display_df['time_new'] = pd.to_datetime(display_df['time_new'])
        display_df['time_dff'] = display_df.apply(lambda x: (x['time_sch'] - x['time_new']).total_seconds()/ 60,axis = 1 )
        display_df['time_dff'] = display_df['time_dff'].apply(lambda x: -x if x <= 0 else "diverted" )
        n_diverted = len(display_df.query("time_dff == 'diverted'"))
        print(f"{n_diverted} flights being diverted")
        return display_df
    
    def get_basemodel(self):
        given_data = self.df.copy()
        base_df = self.initial.copy()

        def parse_time_string(timestamp):
            year = timestamp.year
            month = timestamp.month
            day = timestamp.day
            hour = timestamp.hour
            minute = timestamp.minute
            return year, month, day, hour, minute

        year, month, date, hour, min = parse_time_string(self.resumetime)
        flights_in_slot = 0
        for _, flight in given_data.iterrows():
            time_sch_org = flight['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year}-{month}-{date} {hour}:{min}")
            if (time_sch_org - time_sch_new).total_seconds() < 0:
                time_sch = f"{year} {month} {date} {hour:02d}:{min:02d}"
                util = self.compute_util(flight['code'], year, month, date, hour, min)
            else: 
                time_sch = time_sch_org.strftime("%Y %m %d %H:%M")
                util = 0
            base_df.loc[flight['code']] = [time_sch, util]

            # Increment flight count and time slot only if the slot is full
            ori_min = min
            flights_in_slot += 1
            if flights_in_slot >= self.n_runway:
                min = ori_min + self.time_slot_duration
                flights_in_slot = 0
                if min >= 60:
                    hour += 1
                    min -= 60
                    if hour >= 24:
                        date += 1
                        hour -= 24
                ori_min = min

        display_df = self.df.copy()
        display_df = pd.merge(self.df[['code','time_sch','pass_load']], base_df, left_on= "code", right_index = True)
        display_df['time_new'] = pd.to_datetime(display_df['time_new'])
        display_df['time_dff'] = display_df.apply(lambda x: (x['time_sch'] - x['time_new']).total_seconds()/ 60,axis = 1 )
        display_df['time_dff'] = display_df['time_dff'].apply(lambda x: -x if x <= 0 else "diverted" )
        n_diverted = len(display_df.query("time_dff == 'diverted'"))
        print(f"{n_diverted} flights being diverted")
        return display_df

# __________________________
# search problem with alternative u function
class reschedule_custom_u(reschedule_problem):
    """inherit the properties of class reschedule_problem, but takes an additional
    argument`util_f`, a function that has two argment: delay and pass_load used to
    calculate the utility of the flights"""

    def __init__(self, df,util_f, n_runway = 1, disruption_dur = 60,
                timeslot_dur = 5, max_delay = 120, divert_penalty = 1):
        super().__init__(df, n_runway , disruption_dur,
                timeslot_dur, max_delay, divert_penalty)
        self.util_f = util_f


    def compute_util(self, flcode, year, month, date, hour, min):
        """Compute the utility of a given rescheduled flight. This is defined
        as the time difference between the original scheduled time and the 
        new scheduled time"""
        if flcode is None:
            return 0 
        elif year != 1970:
            time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            # compute the time delayed
            delay = time_sch_org - time_sch_new
            delay = delay.reset_index(drop = True)[0].total_seconds() / 60

        elif year == 1970:
            # compute utility of diverted flight
            delay = - self.max_delay * self.divert_penalty

        pass_load = self.df.query("code == @flcode")['pass_load'].values[0]
        util = self.util_f(delay, pass_load)

        return util

#____________________________
# search problem with alternative u function and discount on future U
class reschedule_custom_u_dis(reschedule_custom_u):
    """inherit the properties of `reschedule_custom_u`, taking an
    additional argument `dis_rate` which defines how much 
    we value the uncertainty of flights originally scheduled for later"""

    def __init__(self, df,util_f, n_runway = 1, disruption_dur = 60,
                timeslot_dur = 5, max_delay = 120, divert_penalty = 1, dis_rate = 1.05):
        super().__init__(df, util_f,n_runway , disruption_dur,
                timeslot_dur, max_delay, divert_penalty)
        self.dis_rate = dis_rate

    def compute_util(self, flcode, year, month, date, hour, min):
        """Compute the utility of a given rescheduled flight. This is defined
        as the time difference between the original scheduled time and the 
        new scheduled time"""
        
        if flcode is None:
            return 0 
        elif year != 1970:
            time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            # compute the time delayed
            delay = time_sch_org - time_sch_new
            delay = delay.reset_index(drop = True)[0].total_seconds() / 60

        elif year == 1970:
            # compute utility of diverted flight
            delay = - self.max_delay * self.divert_penalty

        pass_load = self.df.query("code == @flcode")['pass_load'].values[0]
        util = self.util_f(delay, pass_load)
        # compute discounted util
        time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
        time_elapse = time_sch_org.values[0]  - self.resumetime
        time_elapse = time_elapse.total_seconds()
        time_period = time_elapse / 60 / self.time_slot_duration
        dis_util = util / (self.dis_rate ** time_period)

        return dis_util

# ____________________________
# node
class Node:
    """This takes a lot of refeneces to the AMINA class node"""
    def __init__(self, state,parent = None, action = None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 
        if parent:
            self.depth = parent.depth + 1
    def __repr__(self):
        """define printing output of the class type"""
        return "<Node of depth {}>".format(self.depth)
    
    def __lt__(self, node):
        """Compare the depth of the nodes -- for use in 
        sorting the *frontier*"""
        return self.depth < node.depth
    
    def expand(self, problem):
        """reach the nodes reachable"""
        flight_to_assign, flight_diverted = problem.actions(self.state)
        if len(flight_to_assign) == 0:
            # when no flight can be assigned at the time slot
            flight_to_assign.append(list())
        return [self.child_node(problem, action, flight_diverted) 
                  for action in flight_to_assign]

    def child_node(self, problem,action, flight_diverted):
        """ Return a Node object representing the child node        
        """
        parent_state = self.state.copy()
        next_state = problem.result(parent_state, flight_diverted, action)
        # print(next_state)
        next_node = Node(next_state, parent=self, action = action, path_cost = problem.utility(next_state))
        return next_node # node object

    
# ____________________________________________
# bfs graph
def best_first_graph_search(problem):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""

    # adding first node
    node = Node(problem.initial)
    print(f"The airport resumed service at {problem.hour}:{problem.min:02d}")
    iterations = 1
    # applying the goal test when generating the node
    if problem.goal_test(node.state):
        return(iterations, node)

    # expand the frontier based on the priority queue
    # the current best candidate for extension
    frontier = list()
    frontier.append(tuple([node.path_cost,node ]))

    while frontier:
        if iterations % 1000 == 0:
            break_command = input("""1000 iterations operated. Continue or break? 
                                  Type in 'break' to break the loop, otherwise anything else""")
            if break_command is not None:
                if break_command.lower().strip == 'break':
                    break
        # get the next node in the frontier
        node = frontier.pop()[1]
        iterations +=1
        # applying the goal test when expanding the node
        if problem.goal_test(node.state):
            return node.state
        # for every child in the frontier
        for child in node.expand(problem): # child is a node
            frontier.append(tuple([int(child.path_cost),child]))
            frontier.sort(reverse = True) # order from lower cost to higher cost (absolute terms)

    print("No solution is returned by the solver algorithm")
    return None # otherwise return node.state
    
# ____________________________________________
# bfs graph
def breadth_first_search(problem):
    """Search the nodes with the lowest depth scores first.
    """
    # adding first node
    node = Node(problem.initial)
    print(f"The airport resumed service at {problem.hour}:{problem.min:02d}")
    iterations = 1
    # applying the goal test when generating the node
    if problem.goal_test(node.state):
        return(iterations, node)

    solutions = []
    # expand the frontier based on the priority queue
    # the current best candidate for extension
    frontier = list()
    frontier.append(tuple([node.depth,node ]))

    while frontier:
        # get the next node in the frontier
        node = frontier.pop()[1]
        # applying the goal test when expanding the node
        if problem.goal_test(node.state):
            # add the node to the list of dictionary
            solutions.append(tuple([node.path_cost,node]))
            print(f"A terminal state has been reached: total {len(solutions)} solutions")
        else:
            # for every child in the frontier
            for child in node.expand(problem): # child is a node
                print(f"Iteration: {iterations} at depth {child.depth}")
                print((f"Check if the depth is consistent withe the node state."))
                print(f"There should be {len(child.state)/problem.n_runway} timeslot iterated.")
                frontier.append(tuple([int(child.depth),child]))
                frontier.sort(reverse = True) # order from lower cost to higher cost (absolute terms)
                iterations+= 1
            
                if iterations % 1000 == 0:
                    msg = "1000 iterations processed. Continue or break? Type 'break' to break out of the loop"
                    command_input = input("Continue or break?")
                    try:
                        command_input = command_input.lower.strip() 
                    except:
                        command_input = None
                    if command_input != "break":
                        break

    # <return the solution from the list that has the maximum utility>
    # compute the utilityof all the solution yielded
    solutions.sort(reverse = True)
    # return the   
    return solutions

In [54]:
df = pd.read_csv("./data/21DEC2023_AMS_processed.csv", parse_dates = ['time_sch','time_act'])
df = df.sort_values("time_sch").reset_index(drop = True)
df.head()

,time_sch,time_act,code,dest,stat,orig,pass_load,time_diff
0,2023-12-21 00:10:00,2023-12-21 01:19:00,HV 6888 Transavia,Amsterdam,BAGGAGE HANDLED,Reykjavik (KEF),383,4140.0
1,2023-12-21 00:25:00,2023-12-21 00:25:00,HV 5336 Transavia,Amsterdam,BAGGAGE HANDLED,Sharm El Sheikh (SSH),364,0.0
2,2023-12-21 01:00:00,2023-12-21 01:00:00,HV 6676 Transavia,Amsterdam,BAGGAGE HANDLED,Tenerife (TFS),319,0.0
3,2023-12-21 05:45:00,2023-12-21 05:45:00,KL 590 KLM,Amsterdam,BAGGAGE HANDLED,Accra (ACC),238,0.0
4,2023-12-21 05:50:00,2023-12-21 06:44:00,KL 810 KLM,Amsterdam,BAGGAGE HANDLED,Jakarta (CGK),327,3240.0


## Test 1: Uniform Cost Search
In this first case, we will consider:
* a dataframe of size 20
* n_runway = 2
* duration of disruption = 60

We set up the problem using the `reschedule_problem` class which defines the path cost as the time delayed of the flight scheduled at the child node.

In [55]:
# subset the tail of the dataframe
df_subset = df.tail(20).reset_index(drop = True)
df_subset.head()

,time_sch,time_act,code,dest,stat,orig,pass_load,time_diff
0,2023-12-21 22:30:00,2023-12-21 22:30:00,OR 3802 TUI fly,Amsterdam,BAGGAGE HANDLED,Tenerife (TFS),236,0.0
1,2023-12-21 22:35:00,2023-12-21 22:35:00,LH 2310 Lufthansa,Amsterdam,CANCELLED,Munich (MUC),339,0.0
2,2023-12-21 22:35:00,2023-12-21 23:37:00,KL 1032 KLM,Amsterdam,BAGGAGE HANDLED,London Heathrow (LHR),344,3720.0
3,2023-12-21 22:40:00,2023-12-22 01:10:00,AZ 118 ITA Airways,Amsterdam,BAGGAGE ON BELT,Milan Linate (LIN),346,-77400.0
4,2023-12-21 23:00:00,2023-12-21 23:00:00,KL 1706 KLM,Amsterdam,CANCELLED,Madrid (MAD),316,0.0


In [56]:
# instantiate the problem
AMS21_n20_1 = reschedule_problem(df_subset, n_runway = 2)
# solve the problem
AMS21_n20_1.get_basemodel()


0 flights being diverted


,code,time_sch,pass_load,time_new,util,time_dff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:30:00,-60.0,60.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:30:00,-55.0,55.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:35:00,-60.0,60.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:35:00,-55.0,55.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:40:00,-40.0,40.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:40:00,-40.0,40.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:45:00,-45.0,45.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:45:00,-45.0,45.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:50:00,-45.0,45.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:50:00,-35.0,35.0


In [57]:
AMS21_n20_1.solve(best_first_graph_search)
AMS21_n20_1.display()

The airport resumed service at 23:30


0 flights being diverted


,code,time_sch,pass_load,time_new,util,time_dff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:30:00,-60.0,60.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:35:00,-60.0,60.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:30:00,-55.0,55.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:35:00,-55.0,55.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:45:00,-45.0,45.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:45:00,-45.0,45.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:40:00,-40.0,40.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:40:00,-40.0,40.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:50:00,-45.0,45.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:55:00,-40.0,40.0


## Test 2 Breadth-First Search
Here we use the breadth first tree search, assuming that the path cost is the depth of the node.
We can see from this particularly case, there is a significant increase of the runtime. 

In this section, we set up the problem for rescheduling 6 flights only because breadth-first search, in general, has a high time complexity as the algorithm seeks to conduct a complete search of all possible terminal state. As such, we also reduce the time of airport disruption to 20 min.

In [58]:
# subset
df_subset_2 = df.tail(6).reset_index(drop = True)
# instantiate the problem
AMS21_n6_2 = reschedule_problem(df_subset_2, n_runway = 2,max_delay=120, disruption_dur= 20)
# solve the problem
AMS21_n6_2.solve(breadth_first_search)
# return the solution
breadth_fs_sol_2 = AMS21_n6_2.solution

The airport resumed service at 23:45
Iteration: 1 at depth 1
Check if the depth is consistent withe the node state.
There should be 0.5 timeslot iterated.
Iteration: 2 at depth 1
Check if the depth is consistent withe the node state.
There should be 0.5 timeslot iterated.
Iteration: 3 at depth 1
Check if the depth is consistent withe the node state.
There should be 0.5 timeslot iterated.
Iteration: 4 at depth 1
Check if the depth is consistent withe the node state.
There should be 0.5 timeslot iterated.
Iteration: 5 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 6 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 7 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 8 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 9 at dep

In [59]:
# inspect the result
n_sol =len(breadth_fs_sol_2)
print(f"There are in total {n_sol} number of solution.")
# unzip the solution list
max_util = breadth_fs_sol_2[0][0]
sol_max_score = [node for util, node in breadth_fs_sol_2 if util == max_util]
print(f"The maximum utility of all solution is {max_util}, with {len(sol_max_score)} yielding this utility")

There are in total 1612 number of solution.
The maximum utility of all solution is -75.0, with 18 yielding this utility


As expected, the algorithm takes a long time before it returns a solution. With this reduced case of rescheduling only 6 flights, it takes more than 15 seconds for the algorithm to complete.
It returns all 1614 solutions, out of which 12 share the same utility of -80, that is the maximum utility across all the solutions.

In fact, the time complexity for the breath first search algorithm put a lot of stress on the computation power and it is simply not feasible to rely on the algorithm this resolve the scheduling issue when a lot of flights have to be rescheduled: Empirically, rescheduling 10 flights takes more than **20 min**. Due to the limitation of the computational power, we will not be discussing such case in this project.

Nonetheless, this highlight a major drawback of the application of best-first search in a real-world scenario:
* If there is not more flights to reschedule, rescheudling the flights with a simple rule of assigning the earliest slot to the earlist plan would not yield a lot of lost in utility. 
* If there is more flights to reschedule, then the algorith simply cannot return a solution within a feasible time constraint before an airport has to return to operation.

## Test 3 Model Comparison
In this section, let's compare the breath-first search algorith with the uniform-cost search. As discussed in the section above, breadth-first search requires a lot of computation. Hence, we will limit the size of our problem to rescheduling only 5 flights in thhis section for a fair comparison of the solution they return.

### Breadth-First Search

In [60]:
# subset
df_subset_3 = df.tail(6).reset_index(drop = True)
# instantiate the problem
AMS21_n5_3 = reschedule_problem(df_subset_3, n_runway = 1,max_delay=120, disruption_dur= 15)
# solve the problem
AMS21_n5_3.solve(breadth_first_search)
# the breadth first search returns a list of solution(s)
breadth_fs_sol_3 = AMS21_n5_3.solution

The airport resumed service at 23:40
Iteration: 1 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 2 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 3 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 4 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 5 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 6 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 7 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 8 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 9 at dep

In [61]:
# inspect the result
n_sol =len(breadth_fs_sol_3)
print(f"There are in total {n_sol} number of solution.")
# unzip the solution list
max_util = breadth_fs_sol_3[0][0]
sol_max_score = [node for util, node in breadth_fs_sol_3 if util == max_util]
print(f"The maximum utility of all solution is {max_util}, with {len(sol_max_score)} yielding this utility")

There are in total 216 number of solution.
The maximum utility of all solution is -90.0, with 216 yielding this utility


In [62]:
# print the optimal solution
breadth_fs_sol_3[0][1].state

,time_new,util
HV 6114 Transavia,2023 12 21 23:40,-15.0
HV 6902 Transavia,2023 12 21 23:45,-5.0
KL 1608 KLM,2023 12 21 23:50,-25.0
HV 5218 Transavia,2023 12 21 23:55,0.0
HV 5806 Transavia,2023 12 22 00:00,-30.0
HV 5666 Transavia,2023 12 22 00:05,-15.0


### Uniform Cost Search

In [63]:
AMS21_n5_3_2 = reschedule_problem(df_subset_3, n_runway = 1,max_delay=120, disruption_dur= 15)
AMS21_n5_3_2.solve(best_first_graph_search)
AMS21_n5_3_2_result = AMS21_n5_3_2.display()

The airport resumed service at 23:40
0 flights being diverted


In [64]:
AMS21_n5_3_2_result

,code,time_sch,pass_load,time_new,util,time_dff
0,HV 6114 Transavia,2023-12-21 23:25:00,300,2023-12-21 23:45:00,-20.0,20.0
1,KL 1608 KLM,2023-12-21 23:25:00,309,2023-12-21 23:40:00,-15.0,15.0
2,HV 5806 Transavia,2023-12-21 23:30:00,310,2023-12-21 23:50:00,-20.0,20.0
3,HV 6902 Transavia,2023-12-21 23:40:00,316,2023-12-21 23:55:00,-15.0,15.0
4,HV 5666 Transavia,2023-12-21 23:50:00,347,2023-12-22 00:00:00,-10.0,10.0
5,HV 5218 Transavia,2023-12-21 23:55:00,304,2023-12-22 00:05:00,-10.0,10.0


### Base model (First Come First Served)

In [65]:
# Instatiate and return the result
AMS21_n5_3 = reschedule_problem(df_subset_3, n_runway = 1,max_delay=120, disruption_dur= 15)
AMS21_n5_3_basemodel = AMS21_n5_3.get_basemodel()

0 flights being diverted


In [66]:
AMS21_n5_3_basemodel

,code,time_sch,pass_load,time_new,util,time_dff
0,HV 6114 Transavia,2023-12-21 23:25:00,300,2023-12-21 23:40:00,-15.0,15.0
1,KL 1608 KLM,2023-12-21 23:25:00,309,2023-12-21 23:45:00,-20.0,20.0
2,HV 5806 Transavia,2023-12-21 23:30:00,310,2023-12-21 23:50:00,-20.0,20.0
3,HV 6902 Transavia,2023-12-21 23:40:00,316,2023-12-21 23:55:00,-15.0,15.0
4,HV 5666 Transavia,2023-12-21 23:50:00,347,2023-12-22 00:00:00,-10.0,10.0
5,HV 5218 Transavia,2023-12-21 23:55:00,304,2023-12-22 00:05:00,-10.0,10.0


### Model Evaluation

In [67]:
# parse the result
compare_df = pd.merge(breadth_fs_sol_3[0][1].state, AMS21_n5_3_2_result[['code',"time_sch",'time_new','util']],
                left_index = True, right_on = 'code', suffixes = ("_breadth","_uniform"))
compare_df = pd.merge(compare_df, AMS21_n5_3_basemodel[['code','time_new','util']],on = 'code')
compare_df.rename(columns={'time_new': 'time_new_base', 'util': 'util_base'}, inplace=True)


In [68]:
# reorder and print result
compare_df[["code","time_sch",'time_new_breadth', 'util_breadth', 'time_new_uniform',
       'util_uniform','time_new_base','util_base']].sort_values("time_sch")

,code,time_sch,time_new_breadth,util_breadth,time_new_uniform,util_uniform,time_new_base,util_base
0,HV 6114 Transavia,2023-12-21 23:25:00,2023 12 21 23:40,-15.0,2023-12-21 23:45:00,-20.0,2023-12-21 23:40:00,-15.0
2,KL 1608 KLM,2023-12-21 23:25:00,2023 12 21 23:50,-25.0,2023-12-21 23:40:00,-15.0,2023-12-21 23:45:00,-20.0
4,HV 5806 Transavia,2023-12-21 23:30:00,2023 12 22 00:00,-30.0,2023-12-21 23:50:00,-20.0,2023-12-21 23:50:00,-20.0
1,HV 6902 Transavia,2023-12-21 23:40:00,2023 12 21 23:45,-5.0,2023-12-21 23:55:00,-15.0,2023-12-21 23:55:00,-15.0
5,HV 5666 Transavia,2023-12-21 23:50:00,2023 12 22 00:05,-15.0,2023-12-22 00:00:00,-10.0,2023-12-22 00:00:00,-10.0
3,HV 5218 Transavia,2023-12-21 23:55:00,2023 12 21 23:55,0.0,2023-12-22 00:05:00,-10.0,2023-12-22 00:05:00,-10.0


In [69]:
# compare the utility
compare_df[['util_breadth','util_uniform','util_base']].sum()

util_breadth   -90.0
util_uniform   -90.0
util_base      -90.0
dtype: float64

From this simple case, we can see that breadth search is not superior than the uniform cost search for this problem. In fact, this is due to the way we defined the utility function, which is the aggregate time-delayed per flight.

We we were to look at all solutions with the highest utilty returned by breadth-first search, we will see that in this problem set up the order of the flight being assigned actually has no effect on the utility in most cases.

For example, looking a the result directly from above, we see that the solution given by breadth-first search and uniform-cost search seems to be different. However, looking at the result from breadth-first search, we can see that if we were to swap the new time allocated to HV6114 and KL1608, it will still give us a result of the same utility. Therefore, in the following test, we will build up the complexity of the model by considering alternative utility function.

## Test 4
Instead of using the solely the time delayed, let's see if we define utility differently will yield variation in the performance between breadth-first search and uniform-cost search. We will use the `reschedule_custom_u` class to set up the problem. The class object have similar behaviours with the `rescheuld_problem` used above. The only difference is that it takes an additional argument `util_f` which defines the methodology of computing the utility of each rescheduled flight, which is defined as a function of:
* time-delayed
* Passenger load



Recall that the passenger load is a simulated data set that follows a normal distribution:
$$pass\_load ~ N(\mu= 300,\sigma^2=50^2)$$

The utility of each node is an additive function of all the individual time delayed (in minute) for each rescheduled flight calculated as:
$$\max(time\_sch - time\_new, max\_delay)$$

Note that utility takes a range of $(max\_delay,0]$, we can rescale with a function of passenger that increases at a decreasing scale. This weigh based on $pass\_load$ should have a range of $[0,1]$ and increases at a decreasing rate. A **proposed scale**:
$$\arctan(\frac{pass\_load}{200})\frac{2}{\pi}$$

With this function, it is more costly to delay a flight with more passenger than one with less if their originaly scheduled time is the same.

In [70]:
def util_f(delay, pass_load):
    scaled_pass_load = pass_load / 200
    util = delay * np.arctan(scaled_pass_load) /np.pi * 2
    return util


### Bread-First Search

In [71]:
# subset
df_subset_4 = df.tail(6).reset_index(drop = True)
# instantiate the problem
AMS21_n6_4 = reschedule_custom_u(df_subset_4,util_f, n_runway = 1,max_delay=120, disruption_dur= 15)
# solve the problem
AMS21_n6_4.solve(breadth_first_search)
# the breadth first search returns a list of solution(s)
breadth_fs_sol_4 = AMS21_n6_4.solution

# inspect the result
n_sol =len(breadth_fs_sol_4)
print(f"There are in total {n_sol} number of solution.")
# unzip the solution list
max_util = breadth_fs_sol_4[0][0]
sol_max_score = [node for util, node in breadth_fs_sol_4 if util == max_util]
print(f"The maximum utility of all solution is {max_util}, with {len(sol_max_score)} yielding this utility")

The airport resumed service at 23:40
Iteration: 1 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 2 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 3 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 4 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 5 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 6 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 7 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 8 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 9 at dep

Iteration: 177 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 178 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 179 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 180 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 181 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 182 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 183 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 184 at depth 5
Check if the depth is consistent withe the node state.
There should be 5.0 timeslot iterated.
Iteration: 185 at depth 5
Check if the d

In [72]:
# print the optimal solution
breadth_fs_sol_4[0][1].state

,time_new,util
HV 6902 Transavia,2023 12 21 23:40,0.000000
HV 5806 Transavia,2023 12 21 23:45,-9.528576
HV 5666 Transavia,2023 12 21 23:50,0.000000
KL 1608 KLM,2023 12 21 23:55,-19.029023
HV 5218 Transavia,2023 12 22 00:00,-3.147738
HV 6114 Transavia,2023 12 22 00:05,-25.026637


### Uniform Cost Search

In [73]:
# Instatiate and return the result
AMS21_n6_4_2 = reschedule_custom_u(df_subset_4,util_f, n_runway = 1,max_delay=120, disruption_dur= 15)
AMS21_n6_4_2.solve(best_first_graph_search)
AMS21_n6_4_2_result = AMS21_n6_4_2.display()

The airport resumed service at 23:40
0 flights being diverted


### Base model (First Come First Served)

In [74]:
AMS21_n6_4_basemodel = AMS21_n6_4.get_basemodel()

0 flights being diverted


### Model Evaluation

In [75]:
# parse the result
compare_df = pd.merge(breadth_fs_sol_4[0][1].state, AMS21_n6_4_2_result[['code',"time_sch",'time_new','util']],
                left_index = True, right_on = 'code', suffixes = ("_breadth","_uniform"))
compare_df = pd.merge(compare_df, AMS21_n6_4_basemodel[['code','time_new','util']],on = 'code')
compare_df.rename(columns={'time_new': 'time_new_base', 'util': 'util_base'}, inplace=True)
# reorder and print result
compare_df[["code","time_sch",'time_new_breadth', 'util_breadth', 'time_new_uniform',
       'util_uniform','time_new_base','util_base']].sort_values("time_sch")


,code,time_sch,time_new_breadth,util_breadth,time_new_uniform,util_uniform,time_new_base,util_base
3,KL 1608 KLM,2023-12-21 23:25:00,2023 12 21 23:55,-19.029023,2023-12-21 23:40:00,-9.514512,2023-12-21 23:45:00,-12.686015
5,HV 6114 Transavia,2023-12-21 23:25:00,2023 12 22 00:05,-25.026637,2023-12-21 23:45:00,-12.513318,2023-12-21 23:40:00,-9.384989
1,HV 5806 Transavia,2023-12-21 23:30:00,2023 12 21 23:45,-9.528576,2023-12-21 23:50:00,-12.704768,2023-12-21 23:50:00,-12.704768
0,HV 6902 Transavia,2023-12-21 23:40:00,2023 12 21 23:40,0.000000,2023-12-21 23:55:00,-9.611636,2023-12-21 23:55:00,-9.611636
2,HV 5666 Transavia,2023-12-21 23:50:00,2023 12 21 23:50,0.000000,2023-12-22 00:00:00,-6.671354,2023-12-22 00:00:00,-6.671354
4,HV 5218 Transavia,2023-12-21 23:55:00,2023 12 22 00:00,-3.147738,2023-12-22 00:05:00,-6.295477,2023-12-22 00:05:00,-6.295477


In [76]:
# compare the utility
compare_df[['util_breadth','util_uniform','util_base']].sum()

util_breadth   -56.731975
util_uniform   -57.311066
util_base      -57.354240
dtype: float64

From this result, we do see that the uniform-cost search does not yield the optimal solution.

## Test 5
Altervatively, utility function with the passenger weigh defined as follow:
$$\frac{e^{pass\_load} -1}{e^{pass\_load}}$$

This function stills follows the desired property, having a range of $[0,1]$ and increases with a decreasing rate.

In [77]:
def util_f(delay, pass_load):
    scaled_pass_load = pass_load/200
    util = delay * (np.exp(scaled_pass_load) - 1) / np.exp(scaled_pass_load)
    return util

### Breadth First Search

In [78]:
# subset
df_subset_5 = df.tail(6).reset_index(drop = True)
# instantiate the problem
AMS21_n6_5 = reschedule_custom_u(df_subset_5, util_f, n_runway = 1,max_delay=120, disruption_dur= 15)
# solve the problem
AMS21_n6_5.solve(breadth_first_search)
# the breadth first search returns a list of solution(s)
breadth_fs_sol_5 = AMS21_n6_5.solution

# inspect the result
n_sol =len(breadth_fs_sol_5)
print(f"There are in total {n_sol} number of solution.")
# unzip the solution list
max_util = breadth_fs_sol_5[0][0]
sol_max_score = [node for util, node in breadth_fs_sol_5 if util == max_util]
print(f"The maximum utility of all solution is {max_util}, with {len(sol_max_score)} yielding this utility")

The airport resumed service at 23:40
Iteration: 1 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 2 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 3 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 4 at depth 1
Check if the depth is consistent withe the node state.
There should be 1.0 timeslot iterated.
Iteration: 5 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 6 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 7 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 8 at depth 2
Check if the depth is consistent withe the node state.
There should be 2.0 timeslot iterated.
Iteration: 9 at dep

Iteration: 56 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 57 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 58 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 59 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 60 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 61 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 62 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 63 at depth 4
Check if the depth is consistent withe the node state.
There should be 4.0 timeslot iterated.
Iteration: 64 at depth 4
Check if the depth is c

### Uniform Cost Search

In [79]:
# Instatiate and return the result
AMS21_n6_5_2 = reschedule_custom_u(df_subset_5, util_f, n_runway = 1,max_delay=120, disruption_dur= 15)
AMS21_n6_5_2.solve(best_first_graph_search)
AMS21_n6_5_2_result = AMS21_n6_5_2.display()

The airport resumed service at 23:40
0 flights being diverted


### Base model (First Come First Served)

In [80]:
AMS21_n6_5_basemodel = AMS21_n6_5.get_basemodel()

0 flights being diverted


### Model Evaluation

In [81]:
# parse the result
compare_df = pd.merge(breadth_fs_sol_5[0][1].state, AMS21_n6_5_2_result[['code',"time_sch",'time_new','util']],
                left_index = True, right_on = 'code', suffixes = ("_breadth","_uniform"))
compare_df = pd.merge(compare_df, AMS21_n6_5_basemodel[['code','time_new','util']],on = 'code')
compare_df.rename(columns={'time_new': 'time_new_base', 'util': 'util_base'}, inplace=True)
# reorder and print result
compare_df[["code","time_sch",'time_new_breadth', 'util_breadth', 'time_new_uniform',
       'util_uniform','time_new_base','util_base']].sort_values("time_sch")


,code,time_sch,time_new_breadth,util_breadth,time_new_uniform,util_uniform,time_new_base,util_base
3,KL 1608 KLM,2023-12-21 23:25:00,2023 12 21 23:55,-23.600644,2023-12-21 23:40:00,-11.800322,2023-12-21 23:45:00,-15.733763
5,HV 6114 Transavia,2023-12-21 23:25:00,2023 12 22 00:05,-31.074794,2023-12-21 23:45:00,-15.537397,2023-12-21 23:40:00,-11.653048
1,HV 5806 Transavia,2023-12-21 23:30:00,2023 12 21 23:45,-11.816280,2023-12-21 23:50:00,-15.755041,2023-12-21 23:50:00,-15.755041
0,HV 6902 Transavia,2023-12-21 23:40:00,2023 12 21 23:40,0.000000,2023-12-21 23:55:00,-11.910374,2023-12-21 23:55:00,-11.910374
2,HV 5666 Transavia,2023-12-21 23:50:00,2023 12 21 23:50,0.000000,2023-12-22 00:00:00,-8.235998,2023-12-22 00:00:00,-8.235998
4,HV 5218 Transavia,2023-12-21 23:55:00,2023 12 22 00:00,-3.906441,2023-12-22 00:05:00,-7.812881,2023-12-22 00:05:00,-7.812881


In [82]:
# compare the utility
compare_df[['util_breadth','util_uniform','util_base']].sum()

util_breadth   -70.398158
util_uniform   -71.052012
util_base      -71.101103
dtype: float64

## Test 6
Another assumption we can make is that the further away it is from the current time, it is more uncertain to realised. Therefore, the uility of future should have a lower weight compare to the utility of the current weight. We will have to overwrite the `compute_util`.


The discount rate should take the value between 1 - 2. We will use the same utility function as defined from test 6.

In [83]:
def util_f(delay, pass_load):
    scaled_pass_load = pass_load/200
    util = delay * (np.exp(scaled_pass_load) - 1) / np.exp(scaled_pass_load)
    return util

In [84]:
# subset
df_subset_6 = df.tail(20).reset_index(drop = True)
# Instatiate and return the result
AMS21_n20_6 = reschedule_custom_u_dis(df_subset_6, util_f, dis_rate= 1.3, n_runway = 1,max_delay=120, disruption_dur= 30)
AMS21_n20_6.solve(best_first_graph_search)
AMS21_n20_6_result = AMS21_n20_6.display()
# display the result
AMS21_n20_6_result


The airport resumed service at 23:00


0 flights being diverted


,code,time_sch,pass_load,time_new,util,time_dff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:00:00,-100.308997,30.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:10:00,-106.093362,35.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:05:00,-91.442098,30.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:15:00,-82.241530,35.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:25:00,-19.850623,25.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:35:00,-21.190620,35.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:30:00,-21.945770,30.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:20:00,-16.400726,20.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:40:00,-19.694922,35.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:50:00,-13.005929,35.0


In [85]:
# subset
df_subset_6 = df.tail(20).reset_index(drop = True)
# Instatiate and return the result
AMS21_n20_6 = reschedule_custom_u(df_subset_6, util_f, n_runway = 1,
                                  max_delay=120, disruption_dur= 30)
AMS21_n20_6.solve(best_first_graph_search)
AMS21_n20_6_result = AMS21_n20_6.display()
# display the result
AMS21_n20_6_result


The airport resumed service at 23:00
0 flights being diverted


,code,time_sch,pass_load,time_new,util,time_dff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:10:00,-27.708850,40.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:05:00,-24.492023,30.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:00:00,-20.523346,25.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:15:00,-28.795046,35.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:25:00,-19.850623,25.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:40:00,-24.217852,40.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:30:00,-21.945770,30.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:20:00,-16.400726,20.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:35:00,-21.945770,30.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:50:00,-28.574027,35.0


Comparing the result, the first one with discount rate for flights that were originally scheduled for later, and the second one with any discount rate, the one with discount rate are more likely to prioritise earlier flights.

## Test 7
In the previous testing, we did not observe any of the flights being diverted. By assumption of our cases, we state that if flights are overdue for longer than a defined period, the flight should be diverted, for example, due to the insufficiency of fuel for a flight to stay on hold for longer than 120 minutes.

The below set up is just to search for a problem set up such that we can verify the problem will sometimes divert flight.

In [86]:
def util_f(delay, pass_load):
    scaled_pass_load = pass_load/200
    util = delay * (np.exp(scaled_pass_load) - 1) / np.exp(scaled_pass_load)
    return util

In [87]:
# subset new df
df_subset_7 = df.tail(40).reset_index(drop = True)
# Instatiate and return the result
AMS21_7 = reschedule_custom_u_dis(df_subset_7, util_f = util_f, n_runway = 2,
                                  max_delay=120, disruption_dur= 110, dis_rate = 1.01,
                                  divert_penalty= 1.01)
AMS21_7.solve(best_first_graph_search)
AMS21_7_result = AMS21_7.display()

The airport resumed service at 23:35
2 flights being diverted


In [88]:
AMS21_7_result

,code,time_sch,pass_load,time_new,util,time_dff
0,KL 1800 KLM,2023-12-21 21:45:00,204,2023-12-21 23:40:00,-91.525926,115.0
1,KL 1246 KLM,2023-12-21 21:50:00,254,2023-12-21 23:35:00,-93.061218,105.0
2,VY 8311 Vueling,2023-12-21 21:55:00,208,1970-01-01 00:00:00,-95.615669,diverted
3,OR 1572 TUI fly,2023-12-21 21:55:00,340,2023-12-21 23:35:00,-99.728142,100.0
4,EJU 7858 easyJet Europe,2023-12-21 22:00:00,250,2023-12-21 23:45:00,-90.507894,105.0
5,EJU 7846 easyJet Europe,2023-12-21 22:00:00,244,2023-12-21 23:50:00,-93.658262,110.0
6,LH 1002 Lufthansa,2023-12-21 22:05:00,291,2023-12-21 23:45:00,-91.696622,100.0
7,BA 444 British Airways,2023-12-21 22:05:00,372,2023-12-21 23:40:00,-95.944305,95.0
8,EJU 7930 easyJet Europe,2023-12-21 22:10:00,292,2023-12-21 23:55:00,-95.472928,105.0
9,EJU 7832 easyJet Europe,2023-12-21 22:10:00,252,2023-12-22 00:10:00,-101.804605,120.0


## Heuristic Search

Recall that uninformed searches (breadth-first search and uniform cost search) computes the utility (or cost) from the initial state to the current exploring node. The heuristic function, on the other hand, computes the cost from the current exploring node to a terminal node.

With the rescheulding problem, most (if not all) of the terminal states are associate to nodes of the same depth. However, under the uniform-cost search problem, there is the likelyhood that within the frontier there are nodes of different depth. As such, it will still be useful to consider the distance (in tree depth) from the terminal states as part of the heuristic function.

Recalled that an A* search is the same with  Uniform-cost search in terms of the structure. The only thing differing is the utility function that determines which node to explore next. In an A* search, it is evaluated by:
$$f(n) = g(n) +h(n)$$

The uniform-cost search algorithm is by design taking $g(n)$ as a function of time delay, taking a range of $(-\infty,0]$. Intuitivity, it expands the node that has the **highest utility** (i.e. lowest maximum value of the cost). Similarly, we could define **h(n)** such that the algorithm expands the node with the high "expected" utility. Alternatively, this could be understood as closest to a terminal state.

Recall that the priority queue order is determined by path_cost attribute of the node. Hence, we need to overwrite the `utility` method of the problem class.

One simple admissble heuristic is simply the number of flights that has been assigned a slot.

In [89]:
class reschedule_heuristic(reschedule_custom_u_dis):
    def __init__(self, df,util_f, n_runway = 1, disruption_dur = 60,
                timeslot_dur = 5, max_delay = 120, divert_penalty = 1, dis_rate = 1.01,
                h_weight = 1):
        super().__init__(df, util_f,n_runway , disruption_dur,
                timeslot_dur, max_delay, divert_penalty,dis_rate)
        self.h_weight = h_weight

    def utility(self, state:pd.DataFrame):
        """Compute aggregate utility of a given state"""
        agg_cost = state['util'].sum()
        # compute heuristic
        heuristic = len(state.index)
        return agg_cost + self.h_weight * heuristic

In [90]:
def util_f(delay, pass_load):
    scaled_pass_load = pass_load/200
    util = delay * (np.exp(scaled_pass_load) - 1) / np.exp(scaled_pass_load)
    return util

By considering the heuristic function, an A-star search should yield the cost-optimal result. In order to check if the result is cost optimal, we will use the result that was generated in test 4.
Recall that in the fourth test of rescheduling 6 flights, we identified a solution with the highest utility of $-56.71$.breadth-first search. By unifrom cost search, the solution returned has a utility of $-57.31$. We will replicate the problem set up again, but this time, including the heuristic dunction in determining the order of node to explore.

In [91]:
# subset
df_subset_4 = df.tail(6).reset_index(drop = True)
# instantiate the problem
AMS21_n6_8 = reschedule_heuristic(df_subset_4,util_f, n_runway = 1,max_delay=120, disruption_dur= 15,
                                  dis_rate= 1)
# solve the problem
AMS21_n6_8.solve(best_first_graph_search)
# the breadth first search returns a list of solution(s)
sol_8 = AMS21_n6_8.solution
sol_8

The airport resumed service at 23:40


,time_new,util
KL 1608 KLM,2023 12 21 23:40,-11.800322
HV 6114 Transavia,2023 12 21 23:45,-15.537397
HV 5806 Transavia,2023 12 21 23:50,-15.755041
HV 6902 Transavia,2023 12 21 23:55,-11.910374
HV 5666 Transavia,2023 12 22 00:00,-8.235998
HV 5218 Transavia,2023 12 22 00:05,-7.812881


In [92]:
sol_8['util'].sum()

-71.052011906694

Comparing the solution of the A* search with other search, it seems that the A-star seach algorithm does not return the optimal solution. Note that in the redblob game example and the route search problem, there is only one terminal state. On the contrary, the rescheduling problem has more than one solution. Therefore, the heuristic function cannot effectively encourage the what was effectively a uniform-cost search agent to explore other shallow nodes, which has a lower utility.

# CSP: Min-conflicts Local Search

The `FlightSchedulerCSP` Class
- Initializes the flight scheduler model with the necessary parameters and calls the `CSP` class constructor.
- Parameters:
  - `flights_df`: DataFrame containing flight details.
  - `disruption_level`: Minutes of delay caused by a disruption.
  - `num_runways`: Number of available runways.
  - `neighbor_window`: The time window in minutes to consider flights as neighbors.
  - `time_slot`: The interval in minutes for generating time slots.
- It also initializes a special 'diverted' slot for handling unscheduled flights.

Min-Conflicts Search Functions

`min_conflicts` Function
- A hill-climbing algorithm to solve the CSP. It iteratively tries to reduce the number of conflicts in the current assignment.
- Includes a conflict resolution parameter ('early', 'force', 'divert').
- Returns the final assignment or diverts unresolved flights.

`min_conflicts_value` Function
- Chooses the best value (time slot) for a variable (flight) that results in the least number of conflicts.
- If there's a tie, the earliest time slot is selected. If no valid time slot is found, it resorts to the diverted slot.

Summary
- This custom CSP model is designed to handle a flight scheduling problem.
- It manages constraints like runway capacity and disruption times, and it includes a mechanism to handle flights that can't be scheduled within the given constraints by diverting them.
- The min-conflicts search algorithm is used to find a near-optimal solution, considering the constraints and the nature of the problem.
- The model is flexible enough to handle various numbers of flights, runways, and disruption scenarios.

In [93]:
from csp_problem import *

In [94]:
df['delay'] = (df.time_act - df.time_sch).dt.seconds / 60 # in minutes
df_subset_6 = df.tail(20).reset_index(drop = True)

In [95]:
# %pip install Jinja2

In [96]:
sample = df_subset_6.copy()
flight_schedule_csp = FlightSchedulerCSP(sample, disruption_level=30, num_runways=1, neighbor_window=10, time_slot=5)
solutions = min_conflicts(flight_schedule_csp, resolve_strategy="early")

sample['new_sch'] = solutions.values()
sample['new_delay'] = (sample.new_sch - sample.time_sch).dt.seconds / 60 # in minutes

print(f"Total new delay: {sample.new_delay.sum()}")
print(f"Total diverted flights: {len(sample[sample.new_sch == flight_schedule_csp.diverted_slot])}")
sample[["code", "time_sch", "pass_load", "new_sch", "delay", "new_delay"]].head(15)

Total new delay: 745.0
Total diverted flights: 0


,code,time_sch,pass_load,new_sch,delay,new_delay
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:00:00,0.0,30.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:05:00,0.0,30.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:10:00,62.0,35.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:15:00,150.0,35.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:30:00,0.0,30.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:35:00,0.0,35.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:40:00,52.0,40.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:45:00,54.0,45.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:50:00,0.0,45.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:45:00,76.0,30.0


In [97]:
data = sample[["code", "time_sch", "pass_load", "new_sch", "delay", "new_delay"]].head(15)
col_names = ["FlightID", "Departure", "Passengers", "NewDep", "Delay", "EstDelay"]
data.columns = col_names

styler = data.style
styler = (
    styler.format(subset="Delay", precision=1)
       .format(subset="EstDelay", precision=1)
       .hide(level=0, axis=0)
       )

table = styler.to_latex(
    caption="Estimated Schedule",
    position_float="centering",
    hrules=True
)

print(table)

\begin{table}
\centering
\caption{Estimated Schedule}
\begin{tabular}{llrlrr}
\toprule
FlightID & Departure & Passengers & NewDep & Delay & EstDelay \\
\midrule
OR 3802 TUI fly & 2023-12-21 22:30:00 & 236 & 2023-12-21 23:00:00 & 0.0 & 30.0 \\
LH 2310 Lufthansa & 2023-12-21 22:35:00 & 339 & 2023-12-21 23:05:00 & 0.0 & 30.0 \\
KL 1032 KLM & 2023-12-21 22:35:00 & 344 & 2023-12-21 23:10:00 & 62.0 & 35.0 \\
AZ 118 ITA Airways & 2023-12-21 22:40:00 & 346 & 2023-12-21 23:15:00 & 150.0 & 35.0 \\
KL 1706 KLM & 2023-12-21 23:00:00 & 316 & 2023-12-21 23:30:00 & 0.0 & 30.0 \\
KL 980 KLM & 2023-12-21 23:00:00 & 186 & 2023-12-21 23:35:00 & 0.0 & 35.0 \\
KL 1118 KLM & 2023-12-21 23:00:00 & 263 & 2023-12-21 23:40:00 & 52.0 & 40.0 \\
KL 1434 KLM & 2023-12-21 23:00:00 & 343 & 2023-12-21 23:45:00 & 54.0 & 45.0 \\
KL 1136 KLM & 2023-12-21 23:05:00 & 263 & 2023-12-21 23:50:00 & 0.0 & 45.0 \\
HV 5136 Transavia & 2023-12-21 23:15:00 & 339 & 2023-12-21 23:45:00 & 76.0 & 30.0 \\
HV 6886 Transavia & 2023-12-21 

## Comparing to Test 6

In [98]:
# subset
df_subset_6 = df.tail(20).reset_index(drop = True)
# Instatiate and return the result
AMS21_n20_6 = reschedule_custom_u(df_subset_6, util_f, n_runway = 1,
                                  max_delay=90, disruption_dur= 30)
AMS21_n20_6.solve(best_first_graph_search)
AMS21_n20_6_result = AMS21_n20_6.display()

print(f"Total new delay: {AMS21_n20_6_result.time_dff.sum()}")
AMS21_n20_6_result

The airport resumed service at 23:00


0 flights being diverted
Total new delay: 730.0


,code,time_sch,pass_load,time_new,util,time_dff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:10:00,-27.708850,40.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:05:00,-24.492023,30.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:00:00,-20.523346,25.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:15:00,-28.795046,35.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:25:00,-19.850623,25.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:40:00,-24.217852,40.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:30:00,-21.945770,30.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:20:00,-16.400726,20.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:35:00,-21.945770,30.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:50:00,-28.574027,35.0


In [99]:
test = AMS21_n20_6_result.copy()
test['csp_sch'] = sample.new_sch
test['csp_delay'] = sample.new_delay
test['delay_diff'] = test.time_dff - test.csp_delay
print(f"Total delay diff: {test.delay_diff.sum()}")
test

Total delay diff: -15.0


,code,time_sch,pass_load,time_new,util,time_dff,csp_sch,csp_delay,delay_diff
0,OR 3802 TUI fly,2023-12-21 22:30:00,236,2023-12-21 23:10:00,-27.708850,40.0,2023-12-21 23:00:00,30.0,10.0
1,LH 2310 Lufthansa,2023-12-21 22:35:00,339,2023-12-21 23:05:00,-24.492023,30.0,2023-12-21 23:05:00,30.0,0.0
2,KL 1032 KLM,2023-12-21 22:35:00,344,2023-12-21 23:00:00,-20.523346,25.0,2023-12-21 23:10:00,35.0,-10.0
3,AZ 118 ITA Airways,2023-12-21 22:40:00,346,2023-12-21 23:15:00,-28.795046,35.0,2023-12-21 23:15:00,35.0,0.0
4,KL 1706 KLM,2023-12-21 23:00:00,316,2023-12-21 23:25:00,-19.850623,25.0,2023-12-21 23:30:00,30.0,-5.0
5,KL 980 KLM,2023-12-21 23:00:00,186,2023-12-21 23:40:00,-24.217852,40.0,2023-12-21 23:35:00,35.0,5.0
6,KL 1118 KLM,2023-12-21 23:00:00,263,2023-12-21 23:30:00,-21.945770,30.0,2023-12-21 23:40:00,40.0,-10.0
7,KL 1434 KLM,2023-12-21 23:00:00,343,2023-12-21 23:20:00,-16.400726,20.0,2023-12-21 23:45:00,45.0,-25.0
8,KL 1136 KLM,2023-12-21 23:05:00,263,2023-12-21 23:35:00,-21.945770,30.0,2023-12-21 23:50:00,45.0,-15.0
9,HV 5136 Transavia,2023-12-21 23:15:00,339,2023-12-21 23:50:00,-28.574027,35.0,2023-12-21 23:45:00,30.0,5.0


In [100]:
data = test[["code", "time_sch", "csp_sch", "time_dff", "csp_delay"]].head(5)
col_names = ["FlightID", "Departure", "Est. Departure", "TreeDelay", "CSPDelay"]
data.columns = col_names

styler = data.style
styler = (
    styler.format(subset="TreeDelay", precision=1)
       .format(subset="CSPDelay", precision=1)
       .hide(level=0, axis=0)
       )

table = styler.to_latex(
    caption="Estimated Schedule",
    position_float="centering",
    hrules=True
)

print(table)

\begin{table}
\centering
\caption{Estimated Schedule}
\begin{tabular}{lllrr}
\toprule
FlightID & Departure & Est. Departure & TreeDelay & CSPDelay \\
\midrule
OR 3802 TUI fly & 2023-12-21 22:30:00 & 2023-12-21 23:00:00 & 40.0 & 30.0 \\
LH 2310 Lufthansa & 2023-12-21 22:35:00 & 2023-12-21 23:05:00 & 30.0 & 30.0 \\
KL 1032 KLM & 2023-12-21 22:35:00 & 2023-12-21 23:10:00 & 25.0 & 35.0 \\
AZ 118 ITA Airways & 2023-12-21 22:40:00 & 2023-12-21 23:15:00 & 35.0 & 35.0 \\
KL 1706 KLM & 2023-12-21 23:00:00 & 2023-12-21 23:30:00 & 25.0 & 30.0 \\
\bottomrule
\end{tabular}
\end{table}



## Analysis

The modifications made to the FlightSchedulerCSP class, including the addition of a diverted_slot and adjustments to time slot generation and constraints, provides in a custom flight scheduling CSP model. Let's summarize the key aspects of this experiment and compare it with tree-based search algorithms using utility functions.
Summary of the Experiment:

- Custom Time Slot Frequency: The time_slot parameter allows you to set the frequency of time slots. This change enables more precise scheduling.

- Extended Time Slot Range: The time slots are generated up to the last possible time considering the disruption level and the time slot interval. This ensures a comprehensive range for rescheduling flights.

- Diverted Slot for Unsolvable Conflicts: A diverted_slot set to a future date acts as a fallback for flights that cannot be accommodated. This helps in managing unsolvable conflicts.

- Constraints Adjustments: The constraint function now accounts for the number of flights scheduled at a time and compares it with the available runway capacity. It also accommodates the diverted slot.

- Handling of Local Minima: The min-conflicts algorithm includes a check to prevent getting stuck in local minima, ensuring that the algorithm provides a solution.

- Selection of Earliest Time Slot: The algorithm selects the earliest time slot that minimizes conflicts, or if not feasible, assigns the flight to the diverted slot.

Pros of the CSP Implementation:

- Flexibility in Scheduling: Customizable time slot intervals offer more control over the scheduling process.
- Efficient Handling of Conflicts: The ability to divert flights provides a clear solution for otherwise unsolvable scheduling conflicts.
- Adaptability to Various Scenarios: The CSP model can adapt to different runway capacities and disruption levels.
- Prevention of Stagnation: Safeguards against local minima ensure the algorithm always returns a result.

Cons of the CSP Implementation:

- Increased Complexity: The added features and parameters make the model more complex and potentially harder to manage.
- Risk of Suboptimal Solutions: The use of a diverted slot, while practical, might lead to suboptimal scheduling outcomes.
- Dependence on Parameter Settings: The effectiveness of the model heavily relies on the chosen values for parameters like time_slot and neighbor_window.

Comparison with Tree-Based Search Using Utility Functions:

- CSP vs. Tree-Based Search: CSP focuses on finding a solution that satisfies all constraints, while tree-based search algorithms aim to find an optimal solution based on a utility function.
- Handling of Conflicts: CSPs, especially with the addition of the diverted slot, can handle conflicts more flexibly compared to tree-based searches, which might require more complex utility functions to manage similar scenarios.
- Scalability and Performance: Tree-based searches can be more computationally intensive, especially for large problem spaces, whereas CSPs with the min-conflicts heuristic can be more efficient in finding satisfactory solutions.
- Optimality of Solutions: Tree-based searches are more likely to find the most optimal solution, but at the cost of increased computational effort. In contrast, CSPs may quickly find a satisfactory solution but not necessarily the most optimal one.

In summary, the CSP model is capable to handle large state spaces quickly. While it offers a practical approach to managing unsolvable conflicts, it may not always provide the most optimal scheduling compared to tree-based search algorithms focused on maximizing a utility function.